<h1>Casestudy Eric Bühler - Aachen </h1>

In [3]:
# Packages used in this notebook
import requests
from nltk.sentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
import collections
from datetime import datetime, timedelta
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
import numpy as np
import pandas as pd
import configparser
import openai
import json
from datetime import datetime

In [4]:

SCRAPER_API_KEY = '93d624551a355d2f8416326ea4c9e398'  # Replace with your Scraper API key

def get_wg_links(pages=2):
    links = []
    for i in range(pages):
        url = f"https://www.wg-gesucht.de/wg-zimmer-in-Aachen.1.0.1.{i}.html?offer_filter=1&city_id=1&sort_order=0&noDeact=1&categories%5B%5D=0&pagination=1&pu="
        scraper_api_url = f"http://api.scraperapi.com?api_key={SCRAPER_API_KEY}&url={url}"
        response = requests.get(scraper_api_url)
        
        if response.status_code != 200:
            print(f"Failed to retrieve {url}, status code: {response.status_code}")
            print(response.text)
            continue

        # Parse results
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extracting all Anzeigen
        anzeigen_raw = soup.find_all('div', class_='col-sm-8 card_body')
        for anzeige in anzeigen_raw:
            anchor_tag = anzeige.find('a')
            if anchor_tag and 'href' in anchor_tag.attrs:
                links.append("https://www.wg-gesucht.de" + anchor_tag['href'])

    return links

def get_anzeigen_html(links):
    anzeigen_html = []
    for link in links:
        scraper_api_url = f"http://api.scraperapi.com?api_key={SCRAPER_API_KEY}&url={link}"
        response = requests.get(scraper_api_url)
        
        if response.status_code != 200:
            print(f"Failed to retrieve {link}, status code: {response.status_code}")
            print(response.text)
            continue

        # Convert the BeautifulSoup object to a string
        soup = BeautifulSoup(response.text, 'html.parser')
        anzeigen_html.append({'html': str(soup), 'link': link})

    # Save the list of dictionaries to a JSON file
    with open('html_pages.json', 'w', encoding='utf-8') as file:
        json.dump(anzeigen_html, file, ensure_ascii=False, indent=4)

    return pd.DataFrame(anzeigen_html)

def analyze_sentiment(article_text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(article_text)
    return sentiment

def calculate_composite_sentiment(zimmer_text,lage_text,wg_leben_text,sonstiges_text):
    texts = {
        'zimmer': zimmer_text,
        'lage': lage_text,
        'wg_leben': wg_leben_text,
        'sonstiges': sonstiges_text
    }

    sentiments = {}
    valid_compound_scores = []

    for key, text in texts.items():
        if text != 'Unknown':
            sentiment = analyze_sentiment(text)
            sentiments[key] = sentiment
            valid_compound_scores.append(sentiment['compound'])
        else:
            sentiments[key] = {'compound': None, 'neg': None, 'neu': None, 'pos': None}

    if valid_compound_scores:
        overall_sentiment = sum(valid_compound_scores) / len(valid_compound_scores)
    else:
        overall_sentiment = None

    sentiments['overall_sentiment'] = overall_sentiment

    return sentiments

FRATERNITIES=[
    "Studentenverbindung", 
    "Burschenschaft", 
    "Corps", 
    "Landsmannschaft", 
    "Akademische Verbindung", 
    "Studentische Verbindung", 
    "Studentenbund",
    "Studentenverein", 
    "Studentenvereinigung", 
    "Kadergemeinschaft",
    "Akademische Vereinigung", 
    "Studierendenverband", 
    "Akademische Gemeinschaft",
    "Studentenbruderschaft", 
    "Universitätsgesellschaft"
]

def contains_any_words(texts):
    # Convert all texts to lower case for case-insensitive comparison
    lower_texts = [text.lower() for text in texts]
    
    # Convert the words to lower case for case-insensitive comparison
    lower_words = [word.lower() for word in FRATERNITIES]
    
    for text in lower_texts:
        if any(word in text for word in lower_words):
            return True
    return False

def load_anzeigen_html(filename='html_pages.json'):
    with open(filename, 'r', encoding='utf-8') as file:
        anzeigen_html = json.load(file)

    # Convert the HTML strings back to BeautifulSoup objects
    for anzeige in anzeigen_html:
        anzeige['html'] = BeautifulSoup(anzeige['html'], 'html.parser')

    return pd.DataFrame(anzeigen_html)

def get_anzeigen_from_html():
    html_list = load_anzeigen_html()
    anzeigen = []
    errors = 0
    for soup in html_list['html']:
        try:
            titel_element = soup.find('h1', class_='headline headline-detailed-view-title')
            titel = titel_element.get_text().replace('\n', '').strip() if titel_element else 'Unknown'

            wg_groesse_element = soup.find('span', class_='mr5')
            wg_groesse = wg_groesse_element['title'].split('er')[0].split()[0] if (wg_groesse_element and 'title' in wg_groesse_element.attrs) else 'Unknown'

            key_facts = soup.find_all('b', class_='key_fact_value')
            groesse = key_facts[0].get_text().split('m')[0].strip() if key_facts and len(key_facts) > 0 else 'Unknown'
            miete = key_facts[1].get_text().split('€')[0].strip() if key_facts and len(key_facts) > 1 else 'Unknown'

            minor_facts = soup.find_all('span', class_='section_panel_value')
            kaltmiete = minor_facts[0].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 0 else 'Unknown'
            nebenkosten = minor_facts[1].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 1 else 'Unknown'
            sonstige_kosten = minor_facts[2].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 2 else 'Unknown'
            kaution = minor_facts[3].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 3 else 'Unknown'
            abloesevereinb = minor_facts[4].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 4 else 'Unknown'
            frei_ab = minor_facts[5].get_text().strip() if minor_facts and len(minor_facts) > 5 else 'Unknown'

            adresse = soup.find_all('a', href='#mapContainer')
            strasse = adresse[0].get_text().strip().split()[0] if adresse else 'Unknown'
            plz = next((text for text in adresse[0].get_text().strip().split() if re.fullmatch(r'\d{5}', text)), 'Unknown') if adresse else 'Unknown'

            online_seit = 'Unknown'
            green_style = soup.find('b', style='color: #218700;')
            grey_style = soup.find('b', style='color: #898989;')

            if green_style:
                online_seit = green_style.get_text().strip()
            elif grey_style:
                online_seit = grey_style.get_text().strip()

            if "Sekunde" in online_seit or "Sekunden" in online_seit:
                online_seit = 0,0
            elif "Minuten" in online_seit or "Minute" in online_seit:
                minutes = ''.join(filter(str.isdigit, online_seit))
                #online_seit = f"00:{minutes.zfill(2)}"
                online_seit = 0.01 * int(minutes)
            elif "Stunden" in online_seit or "Stunde" in online_seit:
                hours = ''.join(filter(str.isdigit, online_seit))
                #online_seit = f"{hours.zfill(2)}:00"
                online_seit = int(hours)
            elif "Tag" in online_seit or "Tage" in online_seit:
                days = ''.join(filter(str.isdigit, online_seit))
                online_seit = int(days) * 24
                #online_seit = f"{hours}:00"
            else:
                try:
                    date_format = "%d.%m.%Y"
                    online_date = datetime.strptime(online_seit, date_format)
                    current_date = datetime.now()
                    days_ago = (current_date - online_date).days
                    online_seit = int(days_ago)*24
                except ValueError:
                    # Handle the case where the date is not in the expected format
                    online_seit = "Unknown"


            zimmer = soup.find('div', id='freitext_0')
            zimmer = zimmer.find('p').get_text().strip() 
            zimmer = re.sub('[^a-zA-ZäöüÄÖÜß-]', ' ', zimmer) if zimmer else 'Unknown'

            lage = soup.find('div', id='freitext_1')
            lage = lage.find('p').get_text().strip() 
            lage = re.sub('[^a-zA-ZäöüÄÖÜß-]', ' ', lage) if lage else 'Unknown'

            wg_leben = soup.find('div', id='freitext_2')
            wg_leben = wg_leben.find('p').get_text().strip() 
            wg_leben = re.sub('[^a-zA-ZäöüÄÖÜß-]', ' ', wg_leben) if wg_leben else 'Unknown'

            sonstiges = soup.find('div', id='freitext_3')
            sonstiges = sonstiges.find('p').get_text().strip()
            sonstiges = re.sub('[^a-zA-ZäöüÄÖÜß-]', ' ', sonstiges) if sonstiges else 'Unknown'

            fraternity = contains_any_words([zimmer,lage,wg_leben,sonstiges])
            fraternity_likely = (int(wg_groesse)>5) & (int(miete)<500)
         
            sentiment=calculate_composite_sentiment(zimmer,lage,wg_leben,sonstiges)['overall_sentiment']

            link = html_list['link'].iloc[0]

            anzeigen.append({
                'titel': titel,
                'bewohner': int(wg_groesse),
                'groesse': int(groesse),
                'miete': int(miete),
                'strasse': strasse,
                'plz': int(plz),
                'online_seit': online_seit,
                'sentiment': float(sentiment),
                'verbindung': fraternity,
                'verbindung_moeglich': fraternity_likely,
                'zimmer': zimmer,
                'lage': lage,
                'wg_leben': wg_leben,
                'sonstiges': sonstiges,
                'link': link
                
            })

        except Exception as e:
            #print(f"An error occurred: {e}, link: {html_list['link'].iloc[0]}")
            errors += 1
            continue
            # Specify the file path
    file_path = 'anzeigen.csv'
    # Save DataFrame to CSV
    pd.DataFrame(anzeigen).to_csv(file_path, index=False, encoding='utf-8')

    return pd.DataFrame(anzeigen)

#Get new links, State how many pages (20 links per page) => Call with 20 -> 400 listings
#links = get_wg_links(20)

#Get the html
#anzeigen_html = get_anzeigen_html(links)

#Get Dataframe
anzeigen = get_anzeigen_from_html()
display(anzeigen)

In [8]:
# Condition checks
condition_one = (anzeigen['verbindung'] == True) & (anzeigen['verbindung_moeglich'] == False)
condition_two = (anzeigen['verbindung'] == False) & (anzeigen['verbindung_moeglich'] == True)

# Check condition one
rows_with_condition_one = anzeigen[condition_one]
if not rows_with_condition_one.empty:
    print("There is at least one row where 'verbindung' is True and 'verbindung_moeglich' is False.")
else:
    print("There are no rows where 'verbindung' is True and 'verbindung_moeglich' is False.")

num_rows_one = condition_one.sum()
print(f"There are {num_rows_one} rows where 'verbindung' is True and 'verbindung_moeglich' is False.")

# Check condition two
rows_with_condition_two = anzeigen[condition_two]
if not rows_with_condition_two.empty:
    print("There is at least one row where 'verbindung' is False and 'verbindung_moeglich' is True.")
else:
    print("There are no rows where 'verbindung' is False and 'verbindung_moeglich' is True.")

num_rows_two = condition_two.sum()
print(f"There are {num_rows_two} rows where 'verbindung' is False and 'verbindung_moeglich' is True.")


There is at least one row where 'verbindung' is True and 'verbindung_moeglich' is False.
There are 2 rows where 'verbindung' is True and 'verbindung_moeglich' is False.
There is at least one row where 'verbindung' is False and 'verbindung_moeglich' is True.
There are 19 rows where 'verbindung' is False and 'verbindung_moeglich' is True.


In [11]:
import pandas as pd

# Load the CSV data into a DataFrame
df = pd.read_csv('/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/google_earth_engine/ndvi_2020_results.csv')

# Sort the DataFrame by 'mean_ndvi' in descending order
sorted_df = df.sort_values(by='mean_ndvi', ascending=False)

# Display the sorted DataFrame
print("\nSorted DataFrame by Decreasing mean NDVI:")
display(sorted_df)

# Optionally, save the sorted DataFrame to a new CSV file
sorted_df.to_csv('sorted_ndvi_2020_results.csv', index=False)



Sorted DataFrame by Decreasing mean NDVI:


,plz_code,mean_ndvi
6,52076,0.283134
1,52074,0.282947
9,52080,0.239880
3,52078,0.230341
4,52072,0.229945
0,52066,0.207245
2,52070,0.200926
8,52068,0.145891
7,52064,0.135038
5,52062,0.099158


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import re

# Define the necessary columns for the DataFrame
necessary_columns = ['bewohner', 'groesse', 'miete', 'plz']

# Create a DataFrame from the 'anzeigen' list with only the necessary columns
df = pd.DataFrame(anzeigen, columns=necessary_columns)

# Convert numeric columns to numeric types and handle errors
df['bewohner'] = pd.to_numeric(df['bewohner'], errors='coerce')
df['groesse'] = pd.to_numeric(df['groesse'], errors='coerce')
df['miete'] = pd.to_numeric(df['miete'], errors='coerce')
#df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce')

# Drop rows with missing values in numeric columns
df = df.dropna(subset=['bewohner', 'groesse', 'miete'])

# Convert 'plz' to categorical and create dummy variables
df['plz'] = df['plz'].astype('category')
df = pd.get_dummies(df, columns=['plz'], drop_first=True)

# Add the intercept (constant term) for the regression model
df = sm.add_constant(df)

# Round numeric values to a specified number of decimal places
df = df.round({'bewohner': 0, 'groesse': 0, 'miete': 2})

# Print data types and a sample of the DataFrame for debugging
# Prepare the data for regression
X = df.drop(columns=['miete'])  # Exclude dependent variable 'miete'
y = df['miete']

# Ensure all columns used for regression are numeric
X = X.apply(pd.to_numeric, errors='coerce')  # Convert to numeric, forcing non-numeric to NaN
X = X.fillna(0)  # Fill NaNs with 0 or some other value depending on context

# Get variable names (column names)
variable_names = X.columns if hasattr(X, 'columns') else list(df.columns.difference(['miete']))
display(X[0:1])

# Convert to np
X = np.asarray(X)
X = np.array(X , dtype=float)
y = np.asarray(y)
y = np.array(y, dtype=float)

# Fit the model
model = sm.OLS(y, X).fit()



# Print the summary of the regression with custom variable names
print("Regression Summary:")
print(model.summary())  # Coefficients table



,const,bewohner,groesse,plz_52064,plz_52066,plz_52068,plz_52070,plz_52072,plz_52074,plz_52078,plz_52080,plz_52159,plz_52249
0,1.0,11,20,False,False,False,False,False,True,False,False,False,False


Regression Summary:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.424
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     14.09
Date:                Thu, 04 Jul 2024   Prob (F-statistic):           8.40e-22
Time:                        12:08:40   Log-Likelihood:                -1426.6
No. Observations:                 243   AIC:                             2879.
Df Residuals:                     230   BIC:                             2925.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        386.5121     25.860

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import re

# Define the necessary columns for the DataFrame
necessary_columns = ['bewohner', 'groesse', 'miete', 'plz','online_seit','sentiment','verbindung']

# Create a DataFrame from the 'anzeigen' list with only the necessary columns
df = pd.DataFrame(anzeigen, columns=necessary_columns)

# Convert numeric columns to numeric types and handle errors
df['bewohner'] = pd.to_numeric(df['bewohner'], errors='coerce')
df['groesse'] = pd.to_numeric(df['groesse'], errors='coerce')
df['miete'] = pd.to_numeric(df['miete'], errors='coerce')
df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce')*100
df['online_seit'] = pd.to_numeric(df['online_seit'], errors='coerce')
df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce')
df['verbindung'] = pd.to_numeric(df['verbindung'], errors='coerce')

# Drop rows with missing values in numeric columns
df = df.dropna(subset=['bewohner', 'groesse', 'miete'])

# Convert 'plz' to categorical and create dummy variables
df['plz'] = df['plz'].astype('category')
df['verbindung'] = df['verbindung'].astype('category')
df = pd.get_dummies(df, columns=['plz'], drop_first=True)
df = pd.get_dummies(df, columns=['verbindung'], drop_first=True)

# Add the intercept (constant term) for the regression model
df = sm.add_constant(df)

# Round numeric values to a specified number of decimal places
df = df.round({'bewohner': 0, 'groesse': 0, 'miete': 2,'sentiment': 5})

# Print data types and a sample of the DataFrame for debugging
# Prepare the data for regression
X = df.drop(columns=['online_seit'])  # Exclude dependent variable 'online_seit'
y = np.log(df['online_seit'])

# Ensure all columns used for regression are numeric
X = X.apply(pd.to_numeric, errors='coerce')  # Convert to numeric, forcing non-numeric to NaN
X = X.fillna(0)  # Fill NaNs with 0 or some other value depending on context

# Get variable names (column names)
display(X[0:1])

# Convert to np
X = np.asarray(X)
X = np.array(X , dtype=float)
y = np.asarray(y)
y = np.array(y, dtype=float)

# Fit the model
model = sm.OLS(y, X).fit()



# Print the summary of the regression with custom variable names
print("Regression Summary:")
print(model.summary())  # Coefficients table



,const,bewohner,groesse,miete,sentiment,plz_52064,plz_52066,plz_52068,plz_52070,plz_52072,plz_52074,plz_52078,plz_52080,plz_52159,plz_52249,verbindung_True
0,1.0,11,20,265,-40.675,False,False,False,False,False,True,False,False,False,False,True


Regression Summary:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.258
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     5.251
Date:                Thu, 04 Jul 2024   Prob (F-statistic):           5.51e-09
Time:                        12:08:49   Log-Likelihood:                -492.74
No. Observations:                 243   AIC:                             1017.
Df Residuals:                     227   BIC:                             1073.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.3746      0.805